КОРОЧЕ
В этом ноутбуке я пытался обучить ast - трансформер для аудио классификации, но он жрет овер 14гб в текущей конфигурации

In [1]:
DATA_PATH = '../new-data-without_silence'
TRAIN_DATA_PATH = f'{DATA_PATH}/train'
TASK_DATA_PATH = f'{DATA_PATH}/test'
TRAIN_FILENAME = f'{DATA_PATH}/train_gt.csv'
TASK_FILENAME = f'{DATA_PATH}/test.csv'

In [2]:
import pandas as pd
from transformers import AutoFeatureExtractor, ASTForAudioClassification, Trainer, ASTConfig, ASTModel, TrainingArguments, ASTFeatureExtractor
from datasets import Dataset, load_dataset, Audio

/opt/miniconda3/envs/tune-ast/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv(TRAIN_FILENAME, header=None, names=['audio', 'label'])
df['audio'] = TRAIN_DATA_PATH + "/" + df['audio']

dataset = Dataset.from_pandas(df).cast_column("audio", Audio())
sampling_rate = dataset.features["audio"].sampling_rate

[ASTConfig](https://huggingface.co/docs/transformers/model_doc/audio-spectrogram-transformer#transformers.ASTConfig)
Думаю можно подобрать параметра полегче

In [4]:
num_mel_bins = 64
max_length = 512

model_config = ASTConfig(
    hidden_size = 128,
    num_hidden_layers = 4, # 12
    num_attention_heads = 4, # 12
    intermediate_size = 512,
    hidden_act = 'relu',
    hidden_dropout_prob = 0.1,
    attention_probs_dropout_prob = 0.0,
    initializer_range = 0.02,
    layer_norm_eps = 1e-12,
    patch_size = 4,
    qkv_bias = True,
    frequency_stride = 10,
    time_stride = 10,
    max_length = max_length,
    num_mel_bins = num_mel_bins
)

model = ASTForAudioClassification(model_config)
#model = ASTForAudioClassification.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")

feature_extractor = ASTFeatureExtractor(num_mel_bins=num_mel_bins, max_length=max_length)

In [5]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Preprocess the dataset
def preprocess_function(examples):
    audio = examples["audio"]  # Adjust based on your dataset structure
    inputs = feature_extractor(audio["array"], return_tensors='np', feature_size=-1, sampling_rate=16000) # Probably not 16000
    inputs['input_values'] = inputs['input_values'].reshape(max_length, num_mel_bins)
    return inputs

# Metrics computing while fine tuning
def compute_metrics(p):
    predictions, labels = p

    argmax_predictions = np.argmax(predictions, axis=1)

    precision = precision_score(labels, argmax_predictions, zero_division=0)
    recall = recall_score(labels, argmax_predictions, zero_division=0)
    f1 = f1_score(labels, argmax_predictions, zero_division=0)
    acc = accuracy_score(labels, argmax_predictions)

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "accuracy": acc,
    }

In [6]:
encoded_dataset = dataset.map(preprocess_function, remove_columns=["audio"]).train_test_split(test_size=0.3)

Map: 100%|██████████| 8803/8803 [01:10<00:00, 124.66 examples/s]


In [29]:
training_arguments = TrainingArguments(
    output_dir="saves/ast",
    learning_rate=1e3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    # data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/opt/miniconda3/envs/tune-ast/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
                                                  
 25%|██▌       | 386/1544 [02:48<06:05,  3.17it/s]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512}


{'eval_loss': nan, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_f1': 0.0, 'eval_accuracy': 0.6308216584627035, 'eval_runtime': 36.5053, 'eval_samples_per_second': 72.346, 'eval_steps_per_second': 4.547, 'epoch': 1.0}


 32%|███▏      | 500/1544 [03:27<05:52,  2.96it/s]  

{'loss': 6918170592675889.0, 'grad_norm': nan, 'learning_rate': 676.1658031088083, 'epoch': 1.3}


                                                  
 50%|█████     | 772/1544 [05:34<03:33,  3.61it/s]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512}


...

 97%|█████████▋| 1500/1544 [10:16<00:14,  2.98it/s]

{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 28.497409326424872, 'epoch': 3.89}


100%|█████████▉| 1543/1544 [10:31<00:00,  3.47it/s]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512}
                                                   
100%|██████████| 1544/1544 [11:06<00:00,  3.47it/s]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512}
100%|██████████| 1544/1544 [11:06<00:00,  2.32it/s]

{'eval_loss': nan, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_f1': 0.0, 'eval_accuracy': 0.6308216584627035, 'eval_runtime': 35.3534, 'eval_samples_per_second': 74.703, 'eval_steps_per_second': 4.695, 'epoch': 4.0}
{'train_runtime': 666.783, 'train_samples_per_second': 36.966, 'train_steps_per_second': 2.316, 'train_loss': 2240340217835456.2, 'epoch': 4.0}


TrainOutput(global_step=1544, training_loss=2240340217835456.2, metrics={'train_runtime': 666.783, 'train_samples_per_second': 36.966, 'train_steps_per_second': 2.316, 'total_flos': 4081499665072128.0, 'train_loss': 2240340217835456.2, 'epoch': 4.0})

In [30]:
task_df = pd.read_csv(TASK_FILENAME, header=None, names=['audio', 'label'])
task_df['audio'] = TASK_DATA_PATH + "/" + task_df['audio']

task_dataset = Dataset.from_pandas(task_df).cast_column("audio", Audio())
sampling_rate = task_dataset.features["audio"].sampling_rate

In [34]:
task_encoded_dataset = task_dataset.map(preprocess_function, remove_columns=["audio"])

Map: 100%|██████████| 2870/2870 [00:20<00:00, 138.54 examples/s]


In [35]:
pred = trainer.predict(task_encoded_dataset)
pred

100%|██████████| 180/180 [00:36<00:00,  4.91it/s]


PredictionOutput(predictions=array([[nan, nan],
       [nan, nan],
       [nan, nan],
       ...,
       [nan, nan],
       [nan, nan],
       [nan, nan]], dtype=float32), label_ids=None, metrics={'test_runtime': 37.0347, 'test_samples_per_second': 77.495, 'test_steps_per_second': 4.86})

In [33]:
pred.predictions.argmax(axis=1)

array([0, 0, 0, ..., 0, 0, 0])

In [18]:
pred_argmax = np.argmax(pred[0], axis=1)

In [19]:
np.sum(pred_argmax)

0

In [13]:
assert len(pred_argmax) == len(task_df)